In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!git clone https://github.com/eth-sri/ERAN.git

In [0]:
%cd ERAN

### Install M4

In [0]:
!wget ftp://ftp.gnu.org/gnu/m4/m4-1.4.1.tar.gz
!tar -xvzf m4-1.4.1.tar.gz

In [0]:
%cd m4-1.4.1

In [0]:
!./configure

In [0]:
!make
!make install

In [0]:
!cp src/m4 /usr/bin

In [0]:
%cd ..

In [0]:
!rm m4-1.4.1.tar.gz

### Install gmp

In [0]:
!wget https://gmplib.org/download/gmp/gmp-6.1.2.tar.xz
!tar -xvf gmp-6.1.2.tar.xz

In [0]:
%cd gmp-6.1.2

In [0]:
!./configure --enable-cxx

In [0]:
!make
!make install

In [0]:
!make check

In [0]:
%cd ..

In [0]:
!rm gmp-6.1.2.tar.xz

### Install mpfr

In [0]:
!wget https://www.mpfr.org/mpfr-current/mpfr-4.0.2.tar.xz
!tar -xvf mpfr-4.0.2.tar.xz

In [0]:
%cd mpfr-4.0.2

In [0]:
!./configure

In [0]:
!make
!make install

In [0]:
%cd ..

In [0]:
!rm mpfr-4.0.2.tar.xz

### Install ELINA

In [0]:
!git clone https://github.com/eth-sri/ELINA.git

In [0]:
%cd ELINA

In [0]:
!./configure

In [0]:
!make
!make install

In [0]:
%cd ..

### Install Gurobi

In [0]:
!wget https://packages.gurobi.com/9.0/gurobi9.0.0_linux64.tar.gz
!tar -xvf gurobi9.0.0_linux64.tar.gz

In [0]:
%cd gurobi900/linux64/src/build

In [0]:
!sed -ie 's/^C++FLAGS =.*$/& -fPIC/' Makefile
!make

In [0]:
!cp libgurobi_c++.a ../../lib/

In [0]:
%cd ../../

In [0]:
!cp lib/libgurobi90.so /usr/lib
!python3 setup.py install

In [0]:
%cd ../../

### Update environment variables:

In [0]:
%env GUROBI_HOME=/content/ERAN/gurobi900/linux64
gurobi_home = %env GUROBI_HOME
%env PATH
path = %env PATH
path += ':' + gurobi_home
print(path)
%env PATH=$path
ld_library_path = %env LD_LIBRARY_PATH
ld_library_path += ':' + gurobi_home
print(ld_library_path)
%env LD_LIBRARY_PATH=$ld_library_path

### Install DeepG

In [0]:
!rm -r deepg
!git clone https://github.com/eth-sri/deepg.git

In [0]:
%cd deepg/code

In [0]:
!sed s/81/90/ <Makefile >Makefile2
!sed s/build/./ <Makefile2 >Makefile3
!rm Makefile Makefile2
!mv Makefile3 Makefile

In [0]:
!mkdir build
!make shared_object

In [0]:
!cp ./build/libgeometric.so /usr/lib
!cd ../..

In [0]:
!ldconfig


### Install other requirements


In [0]:
%cd ../..

In [0]:
!sed "s/tensorflow/tensorflow<2/" <requirements.txt >requirements2.txt
!rm requirements.txt
!mv requirements2.txt requirements.txt
!pip3 install -r requirements.txt

Test DP-trained MNIST
==

In [0]:
%cd /content/ERAN/tf_verify

In [0]:
limit = [20, 20, 20, 20, 20, 20, 20, 20, 180, 20]

import csv
import os

cts = [0] * 10
r = csv.reader(open('/content/sample_data/mnist_test.csv'))
os.makedirs('/content/mnist_test')
for row in r:
  label = int(row[0])
  if cts[label] < limit[label]:
    csv.writer(open(f'/content/mnist_test/{label}_{cts[label]}', 'w')).writerow(row)
    cts[label] += 1

In [0]:
highest_alpha = 0.05
threshold = 0.0001

def find_smallest_robustness_alpha(tf_filename, test_image_filename, out_filename):
  with open(out_filename, 'a') as out_f:
    tf_pcs = tf_filename.split('/')[-1].split('_')
    truncate_proportion = tf_pcs[1]
    l2_norm_clip = tf_pcs[2]
    noise_multiplier = tf_pcs[3]
    dp_epsilon = tf_pcs[4]
    label = test_image_filename.split('/')[-1].split('_')[0]
    out_f.write(f'{truncate_proportion}, {l2_norm_clip}, {noise_multiplier}, {dp_epsilon}, {label}, {find_smallest_robustness_alpha_recursive(tf_filename, test_image_filename, 0, highest_alpha)}\n')

import subprocess
import shutil

def find_smallest_robustness_alpha_recursive(tf_filename, test_image_filename, lower_alpha, upper_alpha):
  if upper_alpha - lower_alpha < threshold:
    return (upper_alpha + lower_alpha) / 2

  shutil.copyfile(test_image_filename, '/content/ERAN/data/mnist_test.csv')
  result = subprocess.run(f'python3 . --netname "{tf_filename}" --epsilon {(upper_alpha + lower_alpha) / 2} --domain deepzono --dataset mnist',
                          stdout=subprocess.PIPE,
                          stderr=subprocess.PIPE,
                          shell=True)
  if 'Failed' in result.stdout.decode('utf-8'):
    return find_smallest_robustness_alpha_recursive(tf_filename, test_image_filename, lower_alpha, (upper_alpha + lower_alpha) / 2)
  elif 'Verified' in result.stdout.decode('utf-8'):
    return find_smallest_robustness_alpha_recursive(tf_filename, test_image_filename, (upper_alpha + lower_alpha) / 2, upper_alpha)
  else:
    return -1

In [0]:
import os

tf_list = [# LIST OF .TF FILENAMES IN /content/drive/My Drive/cs839004
           ]

start_at = 0
try:
  with open('/content/drive/My Drive/cs839004_results.csv', 'r') as f:
    start_at = len(f.readlines())
except:
  pass
print('Starting at ' + str(start_at))
current = 0

for tf_entry in tf_list:
  with os.scandir('/content/mnist_test') as test_d:
    for test_image_entry in test_d:
      if current < start_at:
        current += 1
        continue
      print(current)
      find_smallest_robustness_alpha('/content/drive/My Drive/cs839004/' + tf_entry, '/content/mnist_test/' + test_image_entry.name, '/content/drive/My Drive/cs839004_results.csv')
      current += 1